In [4]:
import os
from experiement_constants import openai_key, hugging_key
os.environ["OPENAI_API_KEY"] = openai_key
os.environ['HUGGINGFACEHUB_API_TOKEN'] = hugging_key
from langchain_openai import OpenAI

In [5]:
llm = OpenAI(temperature=0.8)


In [6]:
text = "What is IPL?"
print(llm.invoke(text))


IPL stands for Indian Premier League, which is a professional Twenty20 cricket league in India. It was founded by the Board of Control for Cricket in India (BCCI) in 2008 and is played every year in the months of March to May. The league consists of eight teams representing different cities in India, and features some of the world's top cricket players. IPL is known for its high-energy and entertaining cricket matches, with a mix of international and local players, and has gained a huge following and popularity in India and around the world.


In [7]:
#using model from huggingface 
from langchain_community.llms import HuggingFaceHub

llm_hugging = HuggingFaceHub(repo_id="google/flan-t5-large",
               model_kwargs={
                   "temperature":0, "max_length":64
               })


d:\Python\LangChan_traning\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
output = llm_hugging.predict("Can you write a poem about Ai")
print(output)

ai ai ai ai ai ai ai ai ai ai ai ai ai ai ai ai ai ai ai ai ai


In [23]:
##Prompt Templates
from langchain.prompts.prompt import PromptTemplate
from langchain.chains import LLMChain

template = "How many teams play in {league}?"
prompt_template = PromptTemplate(
    input_variables=['league'],
    template=template)

prompt_template.format(league='ipl')

'How many teams play in ipl?'

In [26]:
chain = LLMChain(llm=llm,prompt=prompt_template)
#chain.invoke('ipl')

test_chain = prompt_template|llm  #works same as the above LLMChian code
test_chain.invoke({"league":"ipl"})

'\n\nThere are 8 teams that currently play in the Indian Premier League (IPL). They are:\n\n1. Chennai Super Kings\n2. Delhi Capitals\n3. Kings XI Punjab\n4. Kolkata Knight Riders\n5. Mumbai Indians\n6. Rajasthan Royals\n7. Royal Challengers Bangalore\n8. Sunrisers Hyderabad'

In [11]:
#sequential chain
league_prompt = PromptTemplate(input_variables=['league'],
        template="How many teams play in {league}?")

league_chain = LLMChain(llm=llm,prompt=league_prompt)

top_team_prompt = PromptTemplate(input_variables=['teams'],
        template="which is most successfull team out of {teams}?")

top_chain = LLMChain(llm=llm , prompt=top_team_prompt)


In [12]:
from langchain.chains import SimpleSequentialChain

final_chain = SimpleSequentialChain(
    chains=[league_chain,top_chain]
)

final_chain.invoke("ipl")

{'input': 'ipl',
 'output': '\n\nThe most successful team in the IPL is the Mumbai Indians, having won the tournament a total of 5 times (2013, 2015, 2017, 2019, and 2020). They are closely followed by the Chennai Super Kings, who have won the tournament 3 times (2010, 2011, and 2018). Both teams have consistently performed well and have a strong fan base.'}

In [13]:
from langchain.chains import SequentialChain

league_chain = LLMChain(llm=llm,prompt=league_prompt,output_key="teams")
top_chain = LLMChain(llm=llm , prompt=top_team_prompt,output_key="team")

sequential_chain = SequentialChain(
    chains=[league_chain,top_chain],
    input_variables=['league'],
    output_variables=['teams','team']
)

sequential_chain({"league":'ipl'})

{'league': 'ipl',
 'teams': '\n\nThere are a total of eight teams that play in the Indian Premier League (IPL). These teams are:\n\n1. Chennai Super Kings (CSK)\n2. Delhi Capitals (DC)\n3. Kings XI Punjab (KXIP)\n4. Kolkata Knight Riders (KKR)\n5. Mumbai Indians (MI)\n6. Rajasthan Royals (RR)\n7. Royal Challengers Bangalore (RCB)\n8. Sunrisers Hyderabad (SRH)',
 'team': '\n\nOut of these eight teams, the most successful team would be Mumbai Indians (MI). They have won the IPL title a record five times, in 2013, 2015, 2017, 2019, and 2020. They also hold the record for the most playoff appearances (9) and the most wins in the IPL (120). MI is also the current defending champions, having won the 2020 season. '}

In [15]:
#chatmodels using chatOpenAi

from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage,SystemMessage,AIMessage

llm2 = ChatOpenAI(temperature=0.7 , model='gpt-3.5-turbo')




In [17]:
message = [
    SystemMessage(content="You are a comedian Ai assistant"),
    HumanMessage(content="Please provide some comedy punchlines on Ai")
]

llm2.invoke(message)

AIMessage(content='1. "Why did the AI break up with its computer? It couldn\'t handle its emotional baggage!"\n2. "I asked my AI assistant to tell me a joke. It replied, \'I\'m sorry, I\'m not programmed for humor. But I can Google some for you!\'"\n3. "Why did the AI go to therapy? It had too many unresolved \'byte\'-sues!"\n4. "I tried teaching my AI how to dance. Now it just keeps doing the robot!"\n5. "I told my AI to stop making dad jokes. Now it just replies, \'I can\'t help it, it\'s in my data!\'"\n6. "My AI assistant is so smart, it finished my crossword puzzle in record time. Turns out it just Googled all the answers!"\n7. "Why did the AI go to the comedy club? To work on its stand-up algorithms!"\n8. "I asked my AI to make me breakfast. It replied, \'I\'m great at making toast. Just don\'t ask me to poach an egg, that\'s over-easy for me!\'"\n9. "I told my AI to stop being so sarcastic. Now it just responds, \'Oh, sure, because humans never use sarcasm!\'"\n10. "My AI keeps 

In [19]:
#prompt template - output parser - llm 
#output parser -> if i want to modify an output of an llm beforehand for that its use
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser


In [21]:
class CommaSepatatedOutput(BaseOutputParser):
    def parse(self,text:str):
        return text.strip().split(",")
    
system_template = "you are a helpful assistant, when the user gives any input you should generate 5 words synonum commma sepatated"
human_template = "{text}"

chat_prom = ChatPromptTemplate.from_messages(
    [
        ("system",system_template),
        ("human",human_template)
    ]
)

chain = chat_prom|llm2|CommaSepatatedOutput()

In [22]:
chain.invoke({"text":'smile'})

['grin', ' beam', ' smirk', ' laugh', ' simper']